# 📘 Syracuse University iSchool OPT Research Task 05 — Programmatic Verification (2021 vs 2024)

This notebook verifies the Q&A analysis of Syracuse Football’s 2021 and 2024 seasons.

It checks the numbers from the **first-page cumulative stats PDFs**, which I **typed manually into Python dictionaries** (`season_2021` and `season_2024`) at the top of this notebook.

> ⚠️ **No file reads here** — the input is only the two dictionaries. If the PDFs differ, update the dictionaries and re-run.

The notebook:
1. Computes each statistic difference (yards, points, 3rd downs, red zone, turnovers, etc.).
2. Prints **True/False** for each LLM claim check (no exceptions thrown).
3. Shows **descriptive stats** for the two seasons, including W–L and Win% splits.
4. Produces simple bar charts.

LLMs (ChatGPT GPT-4 and GPT-5) provided the narrative outputs. This notebook is the numerical source of truth.

In [ ]:
# --- Typed values (edit here if your PDFs differ) ---
season_2021 = {
    "games": 12,
    "wins": 5,
    "losses": 7,
    "conf_wl": "2-6",
    "nonconf_wl": "3-1",
    "home_wl": "3-3",
    "away_wl": "2-4",
    "neutral_wl": "0-0",
    "off_total_yds_g": 366.5,
    "off_rush_yds_g": 213.5,
    "off_pass_yds_g": 153.0,
    "off_yds_per_play": 5.7,
    "plays_total": 778,
    "pts_g": 24.9,
    "third_down_pct": 32.9,
    "top": "29:14",
    "rz_score_pct": 76.5,
    "rz_td_pct": 58.8,
    "to_margin": -5,
    "sacks_for": 37,
    "sacks_allowed": 32,
    "off_yds_per_rush": 5.2,
    "def_yds_per_rush_allowed": 3.5,
    "penalty_yds_g": 58.5,
    "att_avg": 32461,
    "att_total": 227224
}

season_2024 = {
    "games": 13,
    "wins": 10,
    "losses": 3,
    "conf_wl": "5-3",
    "nonconf_wl": "5-0",
    "home_wl": "5-2",
    "away_wl": "4-1",
    "neutral_wl": "1-0",
    "off_total_yds_g": 467.6,
    "off_rush_yds_g": 97.6,
    "off_pass_yds_g": 370.0,
    "off_yds_per_play": 6.1,
    "plays_total": 1000,
    "pts_g": 34.1,
    "third_down_pct": 49.0,
    "top": "33:04",
    "rz_score_pct": 85.5,
    "rz_td_pct": 69.4,
    "to_margin": 1,
    "sacks_for": 27,
    "sacks_allowed": 29,
    "off_yds_per_rush": 3.2,
    "def_yds_per_rush_allowed": 5.0,
    "penalty_yds_g": 59.0,
    "att_avg": 39130,
    "att_total": 273913
}

season_2021, season_2024

## 🔧 Helpers

In [ ]:
def plays_per_game(total_plays, games):
    return total_plays / games

def top_to_minutes(top_str):
    m, s = top_str.split(":")
    return int(m) + int(s)/60.0

def almost_equal(a, b, tol=1e-6):
    return abs(a - b) <= tol

def wl_to_tuple(wl):
    w, l = wl.split("-")
    return int(w), int(l)

def win_pct_from_wl(wl):
    w, l = wl_to_tuple(wl)
    return (w / (w + l)) * 100 if (w + l) > 0 else 0.0

def win_pct(wins, losses):
    total = wins + losses
    return (wins / total) * 100 if total > 0 else 0.0

## 📊 Descriptive Stats (W–L, Win% splits, season context)

In [ ]:
import pandas as pd

rows = []
for yr, s in [("2021", season_2021), ("2024", season_2024)]:
    overall_wp = win_pct(s["wins"], s["losses"])
    conf_wp = win_pct_from_wl(s["conf_wl"]) if "conf_wl" in s else None
    nconf_wp = win_pct_from_wl(s["nonconf_wl"]) if "nonconf_wl" in s else None
    home_wp = win_pct_from_wl(s["home_wl"]) if "home_wl" in s else None
    away_wp = win_pct_from_wl(s["away_wl"]) if "away_wl" in s else None
    neutral_wp = win_pct_from_wl(s["neutral_wl"]) if "neutral_wl" in s else None

    rows.append({
        "season": yr,
        "W": s["wins"],
        "L": s["losses"],
        "Win%": round(overall_wp, 1),
        "Conf W-L": s["conf_wl"],
        "Conf Win%": round(conf_wp, 1),
        "NonConf W-L": s["nonconf_wl"],
        "NonConf Win%": round(nconf_wp, 1),
        "Home W-L": s["home_wl"],
        "Home Win%": round(home_wp, 1),
        "Away W-L": s["away_wl"],
        "Away Win%": round(away_wp, 1),
        "Neutral W-L": s["neutral_wl"],
        "Neutral Win%": round(neutral_wp, 1),
        "Plays/Game": round(s["plays_total"]/s["games"], 1),
        "Yards/Game": s["off_total_yds_g"],
        "Points/Game": s["pts_g"],
        "3rd-Down %": s["third_down_pct"],
        "RZ Score %": s["rz_score_pct"],
        "RZ TD %": s["rz_td_pct"]
    })

df_desc = pd.DataFrame(rows).set_index("season")
print("\n=== Descriptive Stats (W–L, Win% splits, core metrics) ===\n")
print(df_desc)

# Season-over-season changes (2024 - 2021) for selected metrics
delta = pd.Series({
    "Win%": df_desc.loc["2024", "Win%"] - df_desc.loc["2021", "Win%"],
    "Conf Win%": df_desc.loc["2024", "Conf Win%"] - df_desc.loc["2021", "Conf Win%"],
    "NonConf Win%": df_desc.loc["2024", "NonConf Win%"] - df_desc.loc["2021", "NonConf Win%"],
    "Home Win%": df_desc.loc["2024", "Home Win%"] - df_desc.loc["2021", "Home Win%"],
    "Away Win%": df_desc.loc["2024", "Away Win%"] - df_desc.loc["2021", "Away Win%"],
    "Plays/Game": df_desc.loc["2024", "Plays/Game"] - df_desc.loc["2021", "Plays/Game"],
    "Yards/Game": df_desc.loc["2024", "Yards/Game"] - df_desc.loc["2021", "Yards/Game"],
    "Points/Game": df_desc.loc["2024", "Points/Game"] - df_desc.loc["2021", "Points/Game"],
    "3rd-Down %": df_desc.loc["2024", "3rd-Down %"] - df_desc.loc["2021", "3rd-Down %"],
    "RZ Score %": df_desc.loc["2024", "RZ Score %"] - df_desc.loc["2021", "RZ Score %"],
    "RZ TD %": df_desc.loc["2024", "RZ TD %"] - df_desc.loc["2021", "RZ TD %"]
}).round(1)

print("\n=== Season-over-Season Changes (2024 minus 2021) ===\n")
print(delta)

## ✅ Claim Checks (True/False)

In [ ]:
# Q1: Identity shift (rush vs pass)
rush_drop = season_2021["off_rush_yds_g"] - season_2024["off_rush_yds_g"]
pass_rise = season_2024["off_pass_yds_g"] - season_2021["off_pass_yds_g"]
pass_share_2024 = season_2024["off_pass_yds_g"] / season_2024["off_total_yds_g"]

print("Rush drop (yds/g):", round(rush_drop, 1))
print("Pass rise (yds/g):", round(pass_rise, 1))
print("2024 pass share (%):", round(pass_share_2024*100, 1))

print("Check: rush_drop == 115.9 ->", almost_equal(round(rush_drop,1), 115.9))
print("Check: pass_rise == 217.0 ->", almost_equal(round(pass_rise,1), 217.0))
print("Check: pass_share ~ 80% (75–85) ->", 75 <= pass_share_2024*100 <= 85)

In [ ]:
# Q2: Production (yards, points, ypp, plays)
yds_gain = season_2024["off_total_yds_g"] - season_2021["off_total_yds_g"]
pts_gain = season_2024["pts_g"] - season_2021["pts_g"]
ypp_gain = season_2024["off_yds_per_play"] - season_2021["off_yds_per_play"]

ppg_2021 = plays_per_game(season_2021["plays_total"], season_2021["games"])
ppg_2024 = plays_per_game(season_2024["plays_total"], season_2024["games"])

print("Yards/g gain:", round(yds_gain, 1))
print("Points/g gain:", round(pts_gain, 1))
print("Yards/play gain:", round(ypp_gain, 1))
print("Plays/g 2021:", round(ppg_2021, 1))
print("Plays/g 2024:", round(ppg_2024, 1))

print("Check: yds_gain == 101.1 ->", almost_equal(round(yds_gain,1), 101.1))
print("Check: pts_gain == 9.2 ->", almost_equal(round(pts_gain,1), 9.2))
print("Check: ypp_gain == 0.4 ->", almost_equal(round(ypp_gain,1), 0.4))
print("Check: plays/g 2021 == 64.8 ->", almost_equal(round(ppg_2021,1), 64.8))
print("Check: plays/g 2024 == 76.9 ->", almost_equal(round(ppg_2024,1), 76.9))

In [ ]:
# Q3: Third downs and TOP
third_delta = season_2024["third_down_pct"] - season_2021["third_down_pct"]
top_2021 = top_to_minutes(season_2021["top"])  # minutes decimal
top_2024 = top_to_minutes(season_2024["top"])  # minutes decimal
top_diff_min = top_2024 - top_2021

print("Third-down delta (pp):", round(third_delta, 1))
print("TOP 2021 (min):", round(top_2021, 2))
print("TOP 2024 (min):", round(top_2024, 2))
print("TOP diff (min):", round(top_diff_min, 2))

print("Check: third_delta == 16.1 ->", almost_equal(round(third_delta,1), 16.1))
print("Check: TOP diff ~ +3.8 min ->", 3.7 <= top_diff_min <= 3.9)

In [ ]:
# Q4: Red zone
rz_score_delta = season_2024["rz_score_pct"] - season_2021["rz_score_pct"]
rz_td_delta = season_2024["rz_td_pct"] - season_2021["rz_td_pct"]

print("RZ score delta (pp):", round(rz_score_delta, 1))
print("RZ TD delta (pp):", round(rz_td_delta, 1))

print("Check: RZ score delta == 9.0 ->", almost_equal(round(rz_score_delta,1), 9.0))
print("Check: RZ TD delta == 10.6 ->", almost_equal(round(rz_td_delta,1), 10.6))

In [ ]:
# Q5: Turnovers and sacks
to_delta = season_2024["to_margin"] - season_2021["to_margin"]
sacks_for_delta = season_2024["sacks_for"] - season_2021["sacks_for"]
sacks_allowed_delta = season_2024["sacks_allowed"] - season_2021["sacks_allowed"]

print("TO margin delta:", to_delta)
print("Sacks for delta:", sacks_for_delta)
print("Sacks allowed delta:", sacks_allowed_delta)

print("Check: TO margin delta == 6 ->", to_delta == 6)
print("Check: sacks_for delta == -10 ->", sacks_for_delta == -10)
print("Check: sacks_allowed delta == -3 ->", sacks_allowed_delta == -3)

In [ ]:
# Q6: Rushing efficiency vs rush defense
off_rush_eff_drop = season_2021["off_yds_per_rush"] - season_2024["off_yds_per_rush"]
def_rush_eff_rise = season_2024["def_yds_per_rush_allowed"] - season_2021["def_yds_per_rush_allowed"]

print("Off yds/rush drop:", round(off_rush_eff_drop, 1))
print("Def yds/rush allowed rise:", round(def_rush_eff_rise, 1))

print("Check: off yds/rush drop == 2.0 ->", almost_equal(round(off_rush_eff_drop,1), 2.0))
print("Check: def yds/rush rise == 1.5 ->", almost_equal(round(def_rush_eff_rise,1), 1.5))

In [ ]:
# Q7: Penalties (no strict assertion — both seasons ~58–59 yds/g)
pen_delta = season_2024["penalty_yds_g"] - season_2021["penalty_yds_g"]
print("Penalty yards/g delta (approx):", round(pen_delta, 1))
print("Check: penalties roughly stable ->", abs(pen_delta) <= 1.0)

## 📉 Simple Visuals (matplotlib)

In [ ]:
import matplotlib.pyplot as plt

labels = ["2021", "2024"]
yds = [season_2021["off_total_yds_g"], season_2024["off_total_yds_g"]]
pts = [season_2021["pts_g"], season_2024["pts_g"]]
plays = [season_2021["plays_total"] / season_2021["games"],
         season_2024["plays_total"] / season_2024["games"]]

plt.figure(figsize=(6,4))
plt.bar(labels, yds)
plt.title("Total Offense (yds/game)")
plt.ylabel("Yards per game")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
plt.bar(labels, pts)
plt.title("Scoring (points/game)")
plt.ylabel("Points per game")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
plt.bar(labels, plays)
plt.title("Tempo (plays/game)")
plt.ylabel("Plays per game")
plt.tight_layout()
plt.show()

## 🧾 Result & Credits
- All LLM claims print **True/False** checks above and align with the typed first-page numbers.
- Descriptive stats summarize W–L and Win% splits (overall, conference, non-conference, home, away, neutral).
- If your PDFs differ, update the dictionaries at the top and re-run.

_Notebook built by the student with assistance from ChatGPT (GPT-4 and GPT-5)._

In [ ]:
print("Notebook built by the student with assistance from ChatGPT (GPT-4 and GPT-5).")